<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we compare doing a task in a distributed environment and on a single CPU (single computer)

In [1]:
!git clone https://github.com/cagBRT/PySpark.git

Cloning into 'PySpark'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 399 (delta 42), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (399/399), 3.37 MiB | 19.16 MiB/s, done.
Resolving deltas: 100% (240/240), done.




---



---



# Setup Spark<br>

If using DataBricks this section will be different

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c8e2e6a27571fb54f493d5f94e71e9a07d60a9803a7771b28e37751cb1a8844e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [4]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [5]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [6]:
sc=spark.sparkContext.getOrCreate()



---



---



We will now get all the works of Shakespeare (a very large file) and perform count and sort tasks.

# Get all of Shakespeare’s works

In [7]:
import time

**Let's find out how long it takes a distributed computer to perform the task of counting all the words in Shakespeare's works**.

In [8]:
start_time = time.time()
#Count the number of words in all of Shakespear's works
Words=sc.textFile("/content/PySpark/shakespeare.txt")
WordsCount=Words.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
WordsCount.count()

--- 1.097320318222046 seconds ---


1418390

**Let's find out how long it takes a *single computer* to perform the task of counting all the words in Shakespeare's works**.

In [9]:
start_time = time.time()
f= open("/content/PySpark/shakespeare.txt", "r")
words_shakes= f.read()
f.close()
word_shakes_python=words_shakes.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(word_shakes_python)

--- 0.10163593292236328 seconds ---


1293935

In [10]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     1.0975432395935059
single compute time:  0.10182547569274902


In [11]:
start_time = time.time()
#Count the number of distinct words
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
DistinctWordsCount.count()

--- 0.040022850036621094 seconds ---


67506

In [12]:
start_time = time.time()
words_unique=set(word_shakes_python)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.2015228271484375 seconds ---


86196

In [13]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.040190696716308594
single compute time:  0.20385193824768066


In [14]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
SortedWordsCount.top(20)

--- 2.0078492164611816 seconds ---


[(517065, ''),
 (23242, 'the'),
 (19540, 'I'),
 (18297, 'and'),
 (15623, 'to'),
 (15544, 'of'),
 (12532, 'a'),
 (10824, 'my'),
 (9576, 'in'),
 (9081, 'you'),
 (7851, 'is'),
 (7531, 'that'),
 (7068, 'And'),
 (6948, 'not'),
 (6722, 'with'),
 (6218, 'his'),
 (6009, 'your'),
 (6002, 'be'),
 (5616, 'for'),
 (5236, 'have')]

In [17]:
#start_time = time.time()
#counted_words=[]
#words_unique=list(words_unique)
#for i in range(len(words_unique)):
#  tuples= (word_shakes_python.count(words_unique[i]), words_unique[i])
#  counted_words.append(tuples)
#print("--- %s seconds ---" % (time.time() - start_time))
#sc_time= time.time() - start_time

--- 1706.7744374275208 seconds ---


In [18]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     2.008025884628296
single compute time:  1706.7746877670288


distributed time:     2.008025884628296<br>
single compute time:  1706.7746877670288